## FHIR Data Load

This notebook loads a set of [Synthea](https://github.com/synthetichealth/synthea) bundles, and writes them to a set of Parquet files for later querying.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("tx-analytics-fhir").getOrCreate()

Extract each resource type generated by Synthea using [Bunsen](https://github.com/cerner/bunsen), and write it to a table within the database.

In [ ]:
from bunsen.r4.bundles import load_from_directory, extract_entry

resources = ['patient',
             'observation',
             'medicationrequest',
             'condition']

bundles = load_from_directory(spark, 'file:///usr/share/staging/fhir')

for resource in resources:
    df = extract_entry(spark, bundles, resource)
    df.write.format('parquet').mode('overwrite').save('file:///usr/share/warehouse/default/' + resource)

Clean up the Spark session when finished.

In [ ]:
spark.stop()